In [69]:
# Carga de datos
import pandas as pd

df_repuestos = pd.read_excel(r"C:\Users\danie\OneDrive\IA\Proyecto_Final_IA\data\Lineas de Mantenimiento - repuestos.xlsx")
df_mant = pd.read_excel(r"C:\Users\danie\OneDrive\IA\Proyecto_Final_IA\data\INFORME DE ESTADO DE COMUNICACIONES NO COMUNICACIONES E INFORMATICA ( Br Bl I).xlsx",header=3)

In [70]:
# Mostrar los datos
df_repuestos.head()

,EFECTO (NNE),TIPO DE FALLA,RESEÑA DE LA FALLA,REPUESTOS MAS PROBABLES,REPUESTOS CASI PROBABLES,REPUESTOS POCO PROBABLES
0,MOTOROLA 5050,rx,El equipo transmite pero no recibe,modulo,transistor pnp,NaN
1,MOTOROLA 5050,rx,El equipo transmite pero no recibe,PTT,NaN,NaN
2,MOTOROLA 5050,rx,El equipo transmite pero no recibe,antena,NaN,NaN
3,MOTOROLA 5050,no enciende,El equipo no enciende,placa de alimentacion,bateria,NaN
4,MOTOROLA 5050,no enciende,El equipo no enciende,pin de carga,NaN,NaN


**Tratamiento de los datos**
a) Borrar los los nombres de las columnas invalidas
B) Reemplazar los NaN por 0 - menos la ultima columna


In [78]:
# reemplazar las columnas no validas por la última columna válida anterior
new_cols = []
for i, col in enumerate(df_mant.columns):
    if "Unnamed" in col:
                new_cols.append(new_cols[-1])
    else:
        new_cols.append(col)

df_mant.columns = new_cols

# Eliminos los NaN todas menos la última
df_mant.iloc[:, :-1] = df_mant.iloc[:, :-1].fillna(0)

In [99]:
df_mant.head()

,N° Ord,NNE,INE,BAL TANDIL,BAL TANDIL,BAL TANDIL,ESC COM BL 1,ESC COM BL 1,ESC COM BL 1,CDO BR BL I,...,JEF AGR AA EJ 601 - EC,JEF AGR AA EJ 601 - EC,JEF AGR AA EJ 601 - EC,G MANT SIS AA 601,G MANT SIS AA 601,G MANT SIS AA 601,TOTAL GENERAL,TOTAL GENERAL,TOTAL GENERAL,INDIQUE FALLAS
0,0,0,0,Cargo,E/S,F/S,Cargo,E/S,F/S,Cargo,...,Cargo,E/S,F/S,Cargo,E/S,F/S,Cargo,E/S,F/S,PRINCIPALES
1,1,3895002526898,EQ DE TEN Y REPL RL 31 P/CABLE TEF CAMP,0,0,0,7,7,0,0,...,0,0,0,0,0,0,8,8,0,NaN
2,2,3895003563937,EJE DE CARRETE DE TEN Y REPL RL 27 D P/CABLE T...,0,0,0,7,7,0,0,...,0,0,0,0,0,0,7,7,0,NaN
3,3,3895005377953,EQ DE TEN Y REPL RL 26 P/CABLE TEF CAMP,0,0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,1,UM (CUENTA MUSEO ) (ESC COM BL 1)
4,4,5805005211320,TELEFONO DE CAMPAÑA PORTATIL TA 1/PT,0,0,0,11,0,11,0,...,0,0,0,0,0,0,11,0,11,UM (CUENTA MUSEO ) (ESC COM BL 1)


In [97]:
# Columnas a excluir
excluir = ['N° Ord', 'NNE', 'INE', 'TOTAL GENERAL', 'INDIQUE FALLAS']

# Columnas únicas sin las excluidas
cols_filtradas = df_mant.columns.drop_duplicates().drop(excluir)

# Mostrar en columnas verticales
#Unidades = df_mant[cols_filtradas].nunique()
#print(Unidades)
print("Elementos Militares:")
for col in cols_filtradas:
    print(col)  

Elementos Militares:
BAL TANDIL
ESC COM BL 1
CDO BR BL I
ESC ICIA BL 1
RC TAN 10
GA BL 1
RC TAN 2
ESC ING BL 1
RI MEC 7
ESC EXPL C BL 1
RC TAN 8
GAA MIX 602
GAA 601
JEF AGR AA EJ 601 - EC
G MANT SIS AA 601


In [154]:
# Usar la primera fila como encabezado y eliminar los nombres originales
df_fallas = df_mant[['NNE', 'INE', 'TOTAL GENERAL', 'INDIQUE FALLAS']]
df_fallas.columns = df_fallas.iloc[0] # Elimino la fila 0
df_fallas = df_fallas.iloc[:-2] # Elimino las dos últimas filas
df_fallas = df_fallas[1:]   # Elimino la fila 1

# Cargo los nombres de las columnas
df_fallas.columns.values[0] = "NNE"
df_fallas.columns.values[1] = "INE"
df_fallas.columns.values[5] = "PRINCIPALES FALLAS"
filas=df_fallas.shape[0]
df_fallas.head(filas)

,NNE,INE,Cargo,E/S,F/S,PRINCIPALES FALLAS
1,3895002526898,EQ DE TEN Y REPL RL 31 P/CABLE TEF CAMP,8,8,0,NaN
2,3895003563937,EJE DE CARRETE DE TEN Y REPL RL 27 D P/CABLE T...,7,7,0,NaN
3,3895005377953,EQ DE TEN Y REPL RL 26 P/CABLE TEF CAMP,1,0,1,UM (CUENTA MUSEO ) (ESC COM BL 1)
4,5805005211320,TELEFONO DE CAMPAÑA PORTATIL TA 1/PT,11,0,11,UM (CUENTA MUSEO ) (ESC COM BL 1)
5,5805005430012,TELEFONO DE CAMPAÑA PORTATIL TA 312 PT,37,20,0,PERILLA DE VOLUMEN CANT: OCHO (08) DETERIORADA...
...,...,...,...,...,...,...
211,SIN CATALOGAR,GRUPO SENSOR REMOTO,1,1,0,NaN
212,SIN CATALOGAR,TRANSCEPTOR BASE/MOVIL KENWOOD TS-480 SAT - HF,1,1,0,NaN
213,SIN CATALOGAR,TRANSCEPTOR PORTATIL HARRIS RF 5800 HH-104 C/G...,1,1,0,NaN
214,SIN CATALOGAR,TRANSCEPTOR MOVIL KENWOOD TK-7360HK-VHF,2,2,0,NaN


In [158]:
for idx, row in df_fallas.iterrows():
    if (row["Cargo"] - row["E/S"]) != row["F/S"] :
       print (f"Error en fila {idx}: {row['Cargo']} - {row['E/S']} = {(row['Cargo'] - row['E/S'])} != {row['F/S']}")


Error en fila 5: 37 - 20 = 17 != 0
Error en fila 23: 20 - 9 = 11 != 6
Error en fila 38: 1 - 0 = 1 != 0
Error en fila 55: 19 - 17 = 2 != 12
Error en fila 57: 14322 - 14322 = 0 != 1
Error en fila 115: 16 - 12 = 4 != 2
Error en fila 155: 35 - 22 = 13 != 14


In [163]:
fallas_normalizadas = (
    df_fallas["PRINCIPALES FALLAS"]
    .str.upper()                       # todo en mayúsculas
    .str.strip()                       # quitar espacios al inicio y final
    .str.replace(r"\s+", " ", regex=True)  # unifica espacios múltiples
    .str.replace(r"\(\s+", "(", regex=True) # elimina espacio después de "("
    .str.replace(r"\s+\)", ")", regex=True) # elimina espacio antes de ")"
)

# Contar frecuencia
conteo_fallas = fallas_normalizadas.value_counts()

print("Fallas declaradas:")
n=1
for falla, cantidad in conteo_fallas.items():
    print(f" {n}) {falla}: {cantidad}")
    n += 1

Fallas declaradas:
 1) UM (CUENTA MUSEO) (ESC COM BL 1): 11
 2) NO ADMITE REPARACIÓN (G MANT SIS AA 601): 3
 3) TRES (3) FALTA OPTIMIZAR PARA USO CORRECTO WIN10 (GA BL 1): 2
 4) 1 F/S SISTEMA ELECTRICO (RC TAN 2): 2
 5) BATERÍAS F/S (JEF AA EJ 601): 2
 6) 1 F/S INOPERABLE (RC TAN 2): 2
 7) OM NRO 2308019 FECHA 31/05/23 U3 SINTONIZADOR DE ANTENA (F/S) (REMITIDO B MANT COM 601): 1
 8) BAT BB4600 NO RETIENE CARGA CANT: 26 (RC TAN 10) - TRES (3) RT F/S - BATERIAS F/S NO RETIENEN CARGA - MICORTELEFONOS F/S CAPSULAS (ESC ING BL 1) - DOCE (12) MICRO TELEFONO/NO ENCIENDEN/NO SINTONIZA/CAJA DE BATERÍA (JEF AGR AA EJ 601) - BATERÍAS REMITIDAS AL B MANT COM 601. PACK DE BATERÌAS 14.4V- PILA NICD 1.2V 1.800MAH X 8 EN CADA PACK. SRE 137 NRO 22/001319 (GAA MIX 602) - 7884 F/S ETAPA DE REGULACIÓN DE TENSIÓN FUERA DE VALOR, BATERÍA BAJO RENDIMIENTO, 7896 F/S BATERÍA BAJO RENDIMIENTO, 7919 F/S BATERÍA BAJO RENDIMIENTO, 7961 E/S BATERÍA BAJO RENDIMIENTO Y MICROTELEFONO H189 INOPERABLE, 7913 F/S BATERÍA 